In [1]:
%env PLATFORM_API_URL=https://studio.dev.epistemix.cloud/v1

env: PLATFORM_API_URL=https://studio.dev.epistemix.cloud/v1


In [2]:
from epx import Job, ModelConfig, SynthPop

# For visualization
import plotly.express as px
import plotly.io as pio

##### NEED TO FIND A REPLACEMENT FOR THIS TEMPLATE CALL ######
from epxexec.visual.utils import default_plotly_template
pio.templates["epistemix"] = default_plotly_template()
pio.templates.default = "epistemix"


# Transaction Model

This FRED model has all agents 18 and older go shopping. The number of stores,  items available, and item prices are specified in `data/item_inventory_TRANSACTIONS.csv`. The job will print out receipts as agents shop at their chosen stores.

In [ ]:
import time
# Define model configuration
transaction_config = ModelConfig(synth_pop=SynthPop("US_2010.v5", ["Loving_County_TX"]),
    start_date = "2022-05-10",
    end_date = "2022-05-11")

# Configure FRED job
transaction_job = Job(
    "transactions.fred",
    config=[transaction_config],
    key="transaction_job",
    fred_version="11.0.1",
    results_dir="results"
)


# Execute job
transaction_job.execute()

# the following loop idles while we wait for the simulation job to finish
start = time.time()
timeout   = 3000 # timeout in seconds
idle_time = 3   # time to wait (in seconds) before checking status again
while str(transaction_job.status) != 'DONE':
    print(f"{transaction_job.status} after {time.time() - start}")
    if time.time() > start + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    time.sleep(idle_time)

str(transaction_job.status)

NOT STARTED after 0.0019378662109375
NOT STARTED after 3.0060946941375732
NOT STARTED after 6.0095765590667725
NOT STARTED after 9.013474225997925
NOT STARTED after 12.016266345977783
NOT STARTED after 15.019960165023804
NOT STARTED after 18.020597219467163
NOT STARTED after 21.02210569381714
NOT STARTED after 24.03022074699402
NOT STARTED after 27.034742832183838
NOT STARTED after 30.038137197494507
NOT STARTED after 33.04178714752197
NOT STARTED after 36.0459349155426
NOT STARTED after 39.04955434799194
NOT STARTED after 42.053614377975464
NOT STARTED after 45.057403326034546
NOT STARTED after 48.060951232910156
NOT STARTED after 51.062084436416626
NOT STARTED after 54.06310987472534
NOT STARTED after 57.06666946411133
NOT STARTED after 60.07113838195801
NOT STARTED after 63.07235527038574
NOT STARTED after 66.07615399360657
NOT STARTED after 69.07972645759583
NOT STARTED after 72.08326578140259
NOT STARTED after 75.08679604530334
NOT STARTED after 78.0903570652008
NOT STARTED after 

## Data outputs

First, let's look at the CSV file that we generated: 

In [ ]:
transactions_df = transaction_job.runs[1].get_csv_output('transactions.csv')
transactions_df

Let's use it to count how many people of each demographic group shopped at each store:

In [ ]:
transactions_df[['id','race','store']].groupby(['race','store']).count()

We can visualize how much was spent at each of the three stores:

In [ ]:
fig = px.histogram(
    transactions_df,
    x='store',
    y='total_spent',
    title="Total spent at each store"
)
fig.update_layout(
    yaxis_title="Total spent ($)",
    xaxis_title="Store number",
    bargap=0.1,
    title_font_size=30
)
fig.show()

And we can look at the shopping habits of each age group:

In [ ]:
fig = px.histogram(
    transactions_df,
    x='age',
    y='total_spent',
    color = 'store',
    title="Total spent by age group"
)
fig.update_layout(
    yaxis_title="Total spent ($)",
    xaxis_title="Age",
    legend_title = "Store No.",
    bargap=0.1,
    title_font_size=30
)
fig.show()